In [1]:
import tensorwrap as tf
tf.__version__

'0.0.0.3'

In [2]:
from tensorwrap import nn

In [3]:
layer = tf.nn.layers.Dense(units = 100000)
layer1 = tf.nn.layers.Dense(units = 1)

In [4]:
x = tf.range(1, 101, dtype=tf.float32)
x /= 100
x = tf.expand_dims(x, axis = 1)

In [5]:
model = nn.Sequential([
    layer, 
    layer1
])

In [6]:
model.compile(
    loss = tf.nn.losses.mse,
    optimizer = tf.nn.optimizers.SGD(),
    metrics = tf.nn.losses.mae
)

In [7]:
%timeit model.fit(x, x + 10, epochs = 100)

Epoch 1|100 
[=========================]    Loss: 209303280.00000     Metric: 12557.72949
Epoch 2|100 
[=========================]    Loss: 92934487507441025024.00000     Metric: 9468122112.00000
Epoch 3|100 
[=========================]    Loss:        inf     Metric: 5396663433478065356800.00000
Epoch 4|100 
[=========================]    Loss:        inf     Metric:        inf
Epoch 5|100 
[=========================]    Loss:        nan     Metric:        nan
Epoch 6|100 
[=========================]    Loss:        nan     Metric:        nan
Epoch 7|100 
[=========================]    Loss:        nan     Metric:        nan
Epoch 8|100 
[=========================]    Loss:        nan     Metric:        nan
Epoch 9|100 
[=========================]    Loss:        nan     Metric:        nan
Epoch 10|100 
[=========================]    Loss:        nan     Metric:        nan
Epoch 11|100 
[=========================]    Loss:        nan     Metric:        nan
Epoch 12|100 
[=============

In [8]:
x.shape

(100, 1)

In [9]:
model.layers[1].trainable_variables

[Array([[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]], dtype=float32),
 Array([nan], dtype=float32)]

In [10]:
layer.trainable_variables

[Array([[nan, nan, nan, ..., nan, nan, nan]], dtype=float32),
 Array([nan, nan, nan, ..., nan, nan, nan], dtype=float32)]

In [11]:
layer1.trainable_variables

[Array([[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]], dtype=float32),
 Array([nan], dtype=float32)]

In [12]:
model.evaluate(x, x + 10)

Epoch 1|1 
[=========================]    Loss:        nan     Metric:        nan


In [13]:
model.predict(x)

Array([[nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
      